In [69]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
from spot.utils import cst, read_file, write_file, SpecialNames
import os
from spot.type_checking import AnnotCollector, AnnotApplier, MypyChecker, AnnotPath, mypy_checker
from spot.type_inference import TypeInfEnv, TypeInfAction
import shutil


if os.getcwd().endswith('scripts'):
    os.chdir("..")

In [71]:
with mypy_checker(".venv/bin/dmypy", "data/code") as checker:
    check_r = checker.check_code_dir()
    print(check_r.num_error_dict)

Daemon started
{'bad_code_1.py': 7, 'bad_code_2.py': 1}
Daemon stopped


In [80]:
# remove `data/temp` if it exists
inference_dir = "data/code_output/inference"
if os.path.exists(inference_dir):
    shutil.rmtree(inference_dir)
if not os.path.exists(inference_dir):
    os.mkdir(inference_dir)
write_file(f"{inference_dir}/env_code_1.py", read_file("data/code/env_code_1.py"))

In [81]:
inf_checker = MypyChecker(".venv/bin/dmypy", inference_dir)
env = TypeInfEnv(inf_checker, f"{inference_dir}/env_code_1.py")
print(env.state)

Daemon started

num_errors: 0
num_to_annot: 11
to_annotate: [AnnotPath('fib.n'), AnnotPath('fib.<return>'), AnnotPath('foo.bar'), AnnotPath('foo.<return>'), AnnotPath('int_add.a'), AnnotPath('int_add.b'), AnnotPath('int_add.<return>'), AnnotPath('int_tripple_add.a'), AnnotPath('int_tripple_add.b'), AnnotPath('int_tripple_add.c'), AnnotPath('int_tripple_add.<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [74]:
env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))
print(env.state)


num_errors: 0
num_to_annot: 10
to_annotate: [AnnotPath('fib.<return>'), AnnotPath('foo.bar'), AnnotPath('foo.<return>'), AnnotPath('int_add.a'), AnnotPath('int_add.b'), AnnotPath('int_add.<return>'), AnnotPath('int_tripple_add.a'), AnnotPath('int_tripple_add.b'), AnnotPath('int_tripple_add.c'), AnnotPath('int_tripple_add.<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n: Any):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b

def int_tripple_add(a, b, c):
    return a + b + c
        


In [75]:
env.reset()

In [76]:
inf_checker.recheck_files("env_code_1.py")

MypyResult(num_errors=0, num_error_dict={}, output_str='Success: no issues found in 2 source files\n')

In [82]:
env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))
print(env.state)


num_errors: 0
num_to_annot: 10
to_annotate: [AnnotPath('fib.<return>'), AnnotPath('foo.bar'), AnnotPath('foo.<return>'), AnnotPath('int_add.a'), AnnotPath('int_add.b'), AnnotPath('int_add.<return>'), AnnotPath('int_tripple_add.a'), AnnotPath('int_tripple_add.b'), AnnotPath('int_tripple_add.c'), AnnotPath('int_tripple_add.<return>')]
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n: Any):
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar):
    return fib(bar)


def int_add(a, b):
    return a + b + "c"

def int_tripple_add(a, b, c):
    return a + b + c
        


In [83]:
env.reset()

while len(env.state.to_annot) > 0:
    env.step(TypeInfAction(env.state.to_annot[0], cst.Name("str")))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n: Any) -> Any:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar: str) -> str:
    return fib(bar)


def int_add(a: str, b: str) -> str:
    return a + b + "c"

def int_tripple_add(a: str, b: str, c: str) -> str:
    return a + b + c
        


In [84]:
env.reset()

while len(env.state.to_annot) > 0:
    env.step(TypeInfAction(env.state.to_annot[0], cst.Name("int")))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n: int) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar: int) -> int:
    return fib(bar)


def int_add(a: int, b: Any) -> int:
    return a + b + "c"

def int_tripple_add(a: int, b: int, c: int) -> int:
    return a + b + c
        


In [87]:
import random

env.reset()

while len(env.state.to_annot) > 0:
    ty = cst.Name(random.choice(["int", "str"]))
    env.step(TypeInfAction(env.state.to_annot[0], ty))

print(env.state)


num_errors: 0
num_to_annot: 0
to_annotate: []
------------------------ code -------------------------------
from typing import Any

# A recursive fibonacci function
def fib(n: Any) -> int:
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

def foo(bar: int) -> int:
    return fib(bar)


def int_add(a: int, b: Any) -> int:
    return a + b + "c"

def int_tripple_add(a: str, b: str, c: str) -> Any:
    return a + b + c
        
